In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch


from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader
from tf_lab.nips.helper import load_shape_net_models_used_by_wu

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
n_pc_samples = 2048
do_training = True
first_time_running = True
z_rotate = True
load_model = False
seed = 42
max_training_epochs = 2000
loss = 'chamfer'
experiment_name = 'wu_classes_rotated' + '_mlp_arch_' + str(n_pc_samples) +  'pts_' + loss

In [8]:
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/wu_classes_rotated_mlp_arch_2048pts_chamfer'

In [6]:
full_pclouds, full_model_names, full_syn_ids = load_shape_net_models_used_by_wu(n_pc_samples, full_pclouds_path)
model_unames = full_model_names + '.' + full_syn_ids
train_data = PointCloudDataSet(full_pclouds, labels=model_unames)

airplane 02691156
4045 files containing complete point clouds were found.
car 02958343
7497 files containing complete point clouds were found.
chair 03001627
6778 files containing complete point clouds were found.
sofa 04256520
3173 files containing complete point clouds were found.
rifle 04090263
2372 files containing complete point clouds were found.
boat 02858304
1137 files containing complete point clouds were found.
table 04379243
8509 files containing complete point clouds were found.


In [9]:
if n_pc_samples == 2048:
     
    encoder_args = {'n_filters': [64, 128, 256, 512, 1024],
                    'filter_sizes' :[1, 1, 1, 1, 1],
                    'strides': [1, 1, 1, 1, 1]}
else:
    assert(False)


if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    
else:
    n_input = [n_pc_samples, 3]
    
    decoder_args = {'layer_sizes': [1024, 2048, np.prod(n_input)] }
    
    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = z_rotate,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args,        
               )

    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)

In [ ]:
if do_training:
    training_stats = []
    training_stats.append(ae.train(train_data, conf))    
    with open(osp.join(conf.train_dir, 'train_stats.txt'), 'a') as fout:
        np.savetxt(fout, np.array(training_stats)[0])

('Epoch:', '0001', 'training time (minutes)=', '1.6203', 'loss=', '0.003133483')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/wu_classes_rotated_mlp_arch_2048pts_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '1.6418', 'loss=', '0.001479707')
('Epoch:', '0003', 'training time (minutes)=', '1.6591', 'loss=', '0.001216393')
('Epoch:', '0004', 'training time (minutes)=', '1.6617', 'loss=', '0.001129711')
('Epoch:', '0005', 'training time (minutes)=', '1.6648', 'loss=', '0.001029625')
('Epoch:', '0006', 'training time (minutes)=', '1.6643', 'loss=', '0.001002645')
('Epoch:', '0007', 'training time (minutes)=', '1.6631', 'loss=', '0.000904394')
('Epoch:', '0008', 'training time (minutes)=', '1.6615', 'loss=', '0.000874362')
('Epoch:', '0009', 'training time (minutes)=', '1.6664', 'loss=', '0.000791103')
('Epoch:', '0010', 'training time (minutes)=', '1.6622', 'loss=', '